In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from collections import Counter

import functools

import math

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

In [20]:
inputFile = "water-treatmennt-original-marked.csv"

dataFrame = pd.read_csv(inputFile, header = 0, sep = ';')
print(dataFrame.shape)
data = dataFrame.values

print('First few data rows:')
print(data[0:10, 0:5])

numSamples = dataFrame.shape[0]
print("Num samples == " + str(numSamples))
numFeatures = dataFrame.shape[1] - 2  #first feature is sampling date and last feature used as class marker
print("Num features == " + str(numFeatures))

regularizationStrength = 10000
learningRate = 0.000001

(527, 40)
First few data rows:
[['D-1/3/90' 44101.0 1.5 7.8 nan]
 ['D-2/3/90' 39024.0 3.0 7.7 nan]
 ['D-4/3/90' 32229.0 5.0 7.6 nan]
 ['D-5/3/90' 35023.0 3.5 7.9 205.0]
 ['D-6/3/90' 36924.0 1.5 8.0 242.0]
 ['D-7/3/90' 38572.0 3.0 7.8 202.0]
 ['D-8/3/90' 41115.0 6.0 7.8 nan]
 ['D-9/3/90' 36107.0 5.0 7.7 215.0]
 ['D-11/3/90' 29156.0 2.5 7.7 206.0]
 ['D-12/3/90' 39246.0 2.0 7.8 172.0]]
Num samples == 527
Num features == 38


In [6]:
Y = dataFrame.iloc[:, -1]  # Class markers
X = dataFrame.iloc[:, 1:-1]  # Features without date and class markers

print(X.iloc[0:10, 0:2])

# featureMaxValues = [0] * numFeatures
# featureMinValues = [0] * numFeatures

# for featureId in range(numFeatures):
#     featureMaxValues[featureId] = np.amax(X[:, featureId])
#     featureMaxValues[featureId] = np.amin(X[:, featureId])

normalizedX = MinMaxScaler().fit_transform(X)
X = pd.DataFrame(normalizedX)

print(X.iloc[0:10, 0:2])

   Q-E      (input flow to plant)   ZN-E     (input Zinc to plant)
0                         44101.0                              1.5
1                         39024.0                              3.0
2                         32229.0                              5.0
3                         35023.0                              3.5
4                         36924.0                              1.5
5                         38572.0                              3.0
6                         41115.0                              6.0
7                         36107.0                              5.0
8                         29156.0                              2.5
9                         39246.0                              2.0
          0         1
0  0.680598  0.041916
1  0.579121  0.086826
2  0.443305  0.146707
3  0.499151  0.101796
4  0.537147  0.041916
5  0.570087  0.086826
6  0.620915  0.176647
7  0.520817  0.146707
8  0.381883  0.071856
9  0.583558  0.056886


In [14]:
def addFeatureForBias(data):
    extendedData = np.zeros((data.shape[0], data.shape[1] + 1))
    extendedData[:, 0:-1] = data
    extendedData[:, -1] = int(1)
    
    return extendedData

# test = pd.DataFrame([[0, 0],
#                      [0, 0]])
# print(addFeatureForBias(test))

X = addFeatureForBias(X)

In [21]:
def computeSoftMarginCost(weights, X, Y):
    # Hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, weights))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = regularizationStrength * (np.sum(distances) / N)
    
    # Soft margin cost function
    cost = 1 / 2 * np.dot(weights, weights) + hinge_loss
    return cost

def computeCostFunctionGradient(weights, xBatch, yBatch):
    # In case of SGD
    if (type(yBatch) == np.float64):
        yBatch = np.array([yBatch])
        xBatch = np.array([xBatch])
        
    distances = 1 - (yBatch * np.dot(xBatch, weights))
    
    weightsDelta = np.zeros(len(weights))
    
    for index, distance in enumerate(distances):
        if (max(0, distance) == 0):
            deltaI = weights
        else:
            deltaI = weights - (regularizationStrength * yBatch[index] * xBatch[index])
            
        weightsDelta += deltaI
        
    weightsDelta /= len(yBatch)
    
    return weightsDelta

def stochasticGradientDescent(features, outputs):
    maxEpochs = 5000
    weights = np.zeros(features.shape[1]) 
    
    for epoch in range(maxEpochs): 
        # To prevent repeating update cycles
#          X, Y = shuffle(features, outputs)
        
        for ind, x in enumerate(X):
            gradientAscent = computeCostFunctionGradient(weights, x, Y[ind])
            weights = weights - (learningRate * gradientAscent)
            
    return weights

In [22]:
print("training started...")
W = stochasticGradientDescent(X, Y)
print("training finished.")
print("weights are: {}".format(W))

training started...
training finished.
weights are: [0.04938529 0.00512588 0.04439552 0.03661628 0.03890413 0.00724016
 0.06354364 0.01177256 0.01717806 0.03242038 0.0254247  0.00557694
 0.0671211  0.00495743 0.01849328 0.04528484 0.03564538 0.04404416
 0.03266971 0.06516236 0.013401   0.02868622 0.0247133  0.01045085
 0.03336055 0.01733265 0.07566528 0.00253521 0.01466067 0.02797244
 0.04056321 0.0852697  0.07347985 0.05733808 0.07931223 0.06709744
 0.08106181 0.10030369 0.10305658 0.10305658]
